# *Installing Required Libraries*

In [1]:
!pip install transformers
!pip install datasets
!pip install accelerate
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manyli

In [ ]:
%cd '/content/drive/MyDrive/'

[Errno 2] No such file or directory: '/content/drive/MyDrive/'
/content


In [ ]:
!unzip /content/data.zip

unzip:  cannot find or open /content/data.zip, /content/data.zip.zip or /content/data.zip.ZIP.


# *Importing libraries*

In [2]:
import json
import wandb
import pandas as pd
import pickle
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import torch
from datasets import Dataset
from datasets import load_dataset
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from transformers import AutoTokenizer, AutoModelForMaskedLM,AutoModelForSequenceClassification
from transformers import TrainerCallback, EarlyStoppingCallback
from transformers import TrainingArguments, Trainer

In [3]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
val_df = pd.read_csv("val.csv")

train_df = train_df.drop(columns=['Num_Tokens','Num_Sentences','corrected_text'])
test_df = test_df.drop(columns=['Num_Tokens','Num_Sentences','corrected_text'])
val_df = val_df.drop(columns=['Num_Tokens','Num_Sentences','corrected_text'])

In [4]:
import pickle
with open('/content/rh-code-mixed-2.pkl', 'rb') as f:
    related_words = pickle.load(f)

In [5]:
train_df['labels'].value_counts()

,count
labels,
0,2319
1,741


In [6]:
val_df['labels'].value_counts()

,count
labels,
0,774
1,247


In [7]:
test_df['labels'].value_counts()

,count
labels,
0,774
1,247


In [8]:
train_df.to_csv('final_train.csv',index=False, encoding='utf-8')
test_df.to_csv('final_test.csv',index=False, encoding='utf-8')
val_df.to_csv('final_val.csv',index=False, encoding='utf-8')

## *Dataset loading as HuggingFace Trainer*

In [9]:
dataset = load_dataset('csv', data_files={'train': "final_train.csv",
                                              'val':"final_val.csv",'test':"final_test.csv"})

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['clean_text', 'labels', 'language_tags'],
        num_rows: 3060
    })
    val: Dataset({
        features: ['clean_text', 'labels', 'language_tags'],
        num_rows: 1021
    })
    test: Dataset({
        features: ['clean_text', 'labels', 'language_tags'],
        num_rows: 1021
    })
})

## Transliterated Word language with Devanagri Hindi scipt and english words using language tags Approach

In [11]:
def transliterated_preprocess(sentences, related_words):
    texts = sentences['clean_text']
    processed_texts = []

    for text in texts:
        words = text.split()
        process_text = []

        for word in words:
            if related_words[word].get('language') == 'HI':
                process_text.append(related_words[word]['h_script'])
            else:
              if related_words[word].get('corrected_word') != None:
                process_text.append(related_words[word]['corrected_word'])
              else:
                process_text.append(word)

        # Join the list back into a string
        processed_texts.append(" ".join(process_text))

    return {"transliterated_skip_eng": processed_texts}

dataset = dataset.map(
    transliterated_preprocess,
    fn_kwargs={'related_words': related_words},
    batched=True
)

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

In [12]:
print(dataset['train'][0]['transliterated_skip_eng'])

hindu तो कुछ है पर तुम तो पूरी दुनिया मई गंदगी मचा रहे हो or bat रही भगवे की तो तुम्हे aba के अबे भगवा ही थे तलवार के दर से सलवार खोल दी उन्होंने इसमें तुम्हारी कोई गलती नहीं है


In [13]:
dataset['train'][0]

{'clean_text': 'hindu to kuch hai par tum to pori duniya mai gandgi macha rhe ho or bat rhi bhagwe ki to tumhe aba ke abe bhagwa hi the talvar ke dar se salvar khol di unhone isme tumhari koi galti nhi hai',
 'labels': 1,
 'language_tags': 'EN, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, EN, EN, HI, HI, HI, HI, HI, EN, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI',
 'transliterated_skip_eng': 'hindu तो कुछ है पर तुम तो पूरी दुनिया मई गंदगी मचा रहे हो or bat रही भगवे की तो तुम्हे aba के अबे भगवा ही थे तलवार के दर से सलवार खोल दी उन्होंने इसमें तुम्हारी कोई गलती नहीं है'}

In [14]:
dataset['val'][0]

{'clean_text': 'hindu rastra ke ruzhan miya bhai ko kast hai chalo hindu is bat pe mast hai',
 'labels': 1,
 'language_tags': 'EN, EN, HI, HI, HI, HI, HI, HI, HI, HI, EN, HI, EN, HI, EN, HI',
 'transliterated_skip_eng': 'hindu rastra के रुझान मिया भाई को कास्ट है चलो hindu इस bat पे mast है'}

In [15]:
dataset['test'][0]

{'clean_text': 'jai hind , in namuno ko bat bhi karni nahi ati hai , inko pension chahiye',
 'labels': 0,
 'language_tags': 'HI, EN, OOV, EN, HI, HI, EN, HI, HI, HI, HI, HI, OOV, HI, EN, HI',
 'transliterated_skip_eng': 'जय hind , in नमूनों को bat भी करनी नहीं अति है , इनको pension चाहिए'}

# For 5000 rows dataset with clean_text

# *Tokenization as per Hing-MBERT*

In [16]:
tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/hing-mbert")
print(len(tokenizer))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/327 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

119547


In [18]:
mbert_dataset = dataset.map(
    lambda example: tokenizer(example['transliterated_skip_eng'],max_length=97,padding='max_length', truncation=True),
    batched=True,
    batch_size=64
)
mbert_dataset = mbert_dataset.remove_columns(["clean_text","language_tags","transliterated_skip_eng"])
mbert_dataset.set_format("torch")

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

In [19]:
mbert_dataset['train'][0]

{'labels': tensor(1),
 'input_ids': tensor([   101,  19911,  11460,  21042,  22022,  10569,  12213,    880,  14070,
          13841,  21042,  96597,  83874,  70219,    867,  52768,  34315,    889,
          22078,  33555,  13220,  10345,  11519,  57203,    888,  19741,  72109,
          10826,  21042,    880,  14070,  45753, 103860,  11357,  10113,  10412,
          49545,  11554,    888,  19741,  28960,  14080,  17798,    880,  11714,
          52884,  10412, 100906,  11072,    898,  11714,  52884,    866,  51140,
          41607,  27640,  53744,    880,  14070,  45753,  42263,  10914,  38207,
            867,  11714,  18406,  16791,  10569,    102,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [20]:
print("Batch Decode:")
print(tokenizer.batch_decode(mbert_dataset['train'][0]['input_ids']))

Batch Decode:
['[CLS]', 'hin', '##du', 'तो', 'कुछ', 'है', 'पर', 'त', '##ु', '##म', 'तो', 'पूरी', 'दुनिया', 'मई', 'ग', '##ंद', '##गी', 'म', '##चा', 'रहे', 'हो', 'or', 'bat', 'रही', 'भ', '##ग', '##वे', 'की', 'तो', 'त', '##ु', '##म्', '##हे', 'ab', '##a', 'के', 'अब', '##े', 'भ', '##ग', '##वा', 'ही', 'थे', 'त', '##ल', '##वार', 'के', 'दर', 'से', 'स', '##ल', '##वार', 'ख', '##ोल', 'दी', 'उन्होंने', 'इसमें', 'त', '##ु', '##म्', '##हार', '##ी', 'कोई', 'ग', '##ल', '##ती', 'नहीं', 'है', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


In [21]:
cls = [tokenizer.cls_token_id]
sep = [tokenizer.sep_token_id]

input_str = dataset['train'][0]['transliterated_skip_eng']
# Tokenization happens in a few steps:
input_tokens = tokenizer.tokenize(input_str)
input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
input_ids_special_tokens = cls + input_ids + sep

decoded_str = tokenizer.decode(input_ids_special_tokens)

print("start:                ", input_str)
print("tokenize:             ", input_tokens)
print("convert_tokens_to_ids:", input_ids)
print("add special tokens:   ", input_ids_special_tokens)
print("--------")
print("decode:               ", decoded_str)

start:                 hindu तो कुछ है पर तुम तो पूरी दुनिया मई गंदगी मचा रहे हो or bat रही भगवे की तो तुम्हे aba के अबे भगवा ही थे तलवार के दर से सलवार खोल दी उन्होंने इसमें तुम्हारी कोई गलती नहीं है
tokenize:              ['hin', '##du', 'तो', 'कुछ', 'है', 'पर', 'त', '##ु', '##म', 'तो', 'पूरी', 'दुनिया', 'मई', 'ग', '##ंद', '##गी', 'म', '##चा', 'रहे', 'हो', 'or', 'bat', 'रही', 'भ', '##ग', '##वे', 'की', 'तो', 'त', '##ु', '##म्', '##हे', 'ab', '##a', 'के', 'अब', '##े', 'भ', '##ग', '##वा', 'ही', 'थे', 'त', '##ल', '##वार', 'के', 'दर', 'से', 'स', '##ल', '##वार', 'ख', '##ोल', 'दी', 'उन्होंने', 'इसमें', 'त', '##ु', '##म्', '##हार', '##ी', 'कोई', 'ग', '##ल', '##ती', 'नहीं', 'है']
convert_tokens_to_ids: [19911, 11460, 21042, 22022, 10569, 12213, 880, 14070, 13841, 21042, 96597, 83874, 70219, 867, 52768, 34315, 889, 22078, 33555, 13220, 10345, 11519, 57203, 888, 19741, 72109, 10826, 21042, 880, 14070, 45753, 103860, 11357, 10113, 10412, 49545, 11554, 888, 19741, 28960, 14080, 17798, 880, 11714,

# *Calculating Weights for Labels with Imbalanced Dataset*

In [22]:
# Define all possible class labels
class_labels = np.unique(mbert_dataset['train']['labels'])

# Calculate class weights
labels = mbert_dataset['train']['labels']
class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=labels.numpy())
class_weights = torch.tensor(class_weights, dtype=torch.float)
print(class_weights)

tensor([0.6598, 2.0648])


In [23]:
class_labels

array([0, 1])

# **Weighted Loss Trainer for Imbalanced Datset**

In [24]:
from transformers import TrainingArguments, Trainer
# Define a custom Trainer class to include class weights
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels").to(model.device)  # Ensure labels are on the same device as model
        inputs = {k: v.to(model.device) for k, v in inputs.items()}  # Ensure all inputs are on the same device as model
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # Compute weighted loss
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(model.device))  # Move class_weights to the same device as model
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# *Function: Compute metrics for all labels*

In [25]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)

    # Additional metrics
    recall_micro = recall_score(labels, predictions, average='micro')
    f1_micro = f1_score(labels, predictions, average='micro')

    recall_macro = recall_score(labels, predictions, average='macro')
    f1_macro = f1_score(labels, predictions, average='macro')

    recall_positive = recall_score(labels, predictions, pos_label=1)
    f1_positive = f1_score(labels, predictions, pos_label=1)

    recall_negative = recall_score(labels, predictions, pos_label=0)
    f1_negative = f1_score(labels, predictions, pos_label=0)

    return {
        'accuracy': accuracy,
        'precision_weighted': precision,
        'recall_weighted': recall,
        'recall_micro': recall_micro,
        'recall_macro': recall_macro,
        'recall_positive': recall_positive,
        'recall_negative': recall_negative,
        'f1_weighted': f1,
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
        'f1_positive': f1_positive,
        'f1_negative': f1_negative
    }

In [26]:
!export CUDA_LAUNCH_BLOCKING=1

In [27]:
model = AutoModelForSequenceClassification.from_pretrained("l3cube-pune/hing-mbert", num_labels=2)


arguments = TrainingArguments(
    output_dir="sample_HingMBert_trainer_5k_skip_eng",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    optim = 'adamw_torch',
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=5e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model='eval_f1_weighted',  # Define the metric for early stopping
    greater_is_better=True,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224,
    report_to="wandb"
)

trainer = WeightedLossTrainer(
    model=model,
    args=arguments,
    train_dataset=mbert_dataset['train'],
    eval_dataset=mbert_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

config.json:   0%|          | 0.00/861 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/712M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/hing-mbert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# *Logger Function for early Stopping and logging*

In [28]:
class LoggingCallback(TrainerCallback):
    def __init__(self, log_path):
        self.log_path = log_path
    # will call on_log on each logging step, specified by TrainerArguement. (i.e TrainerArguement.logginng_step)
    def on_log(self, args, state, control, logs=None, **kwargs):
        _ = logs.pop("total_flos", None)
        if state.is_local_process_zero:
            with open(self.log_path, "a") as f:
                f.write(json.dumps(logs) + "\n")


In [29]:
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=0.01))
trainer.add_callback(LoggingCallback("sample_HingMBert_trainer_5k_skip_eng/log.jsonl"))

In [30]:
# train the model
wandb.init(project="rh3", name="Hing_mBERT_skip_eng2")
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,Recall Micro,Recall Macro,Recall Positive,Recall Negative,F1 Weighted,F1 Micro,F1 Macro,F1 Positive,F1 Negative
1,No log,0.684607,0.760039,0.710286,0.760039,0.760039,0.527480,0.076923,0.978036,0.684976,0.760039,0.497496,0.134276,0.860716
2,No log,0.592600,0.726738,0.768561,0.726738,0.726738,0.698477,0.643725,0.753230,0.740572,0.726738,0.669792,0.532663,0.806920
3,No log,0.574822,0.785504,0.772271,0.785504,0.785504,0.672457,0.453441,0.891473,0.776578,0.785504,0.684341,0.505643,0.863039
4,No log,0.605613,0.790402,0.780970,0.790402,0.790402,0.689470,0.493927,0.885013,0.784546,0.790402,0.698825,0.532751,0.864899
5,No log,0.631227,0.757101,0.781217,0.757101,0.757101,0.714368,0.631579,0.797158,0.766006,0.757101,0.694901,0.557143,0.832659
6,No log,0.670110,0.703232,0.780923,0.703232,0.703232,0.714674,0.736842,0.692506,0.723049,0.703232,0.662682,0.545727,0.779636
7,No log,0.808199,0.762977,0.775375,0.762977,0.762977,0.701705,0.582996,0.820413,0.768206,0.762977,0.691672,0.543396,0.839947


TrainOutput(global_step=336, training_loss=0.5114006769089472, metrics={'train_runtime': 121.2064, 'train_samples_per_second': 252.462, 'train_steps_per_second': 3.96, 'train_loss': 0.5114006769089472, 'epoch': 7.0})

In [31]:
mbert_results = trainer.evaluate()

In [32]:
print(mbert_results)

{'eval_loss': 0.605612576007843, 'eval_accuracy': 0.7904015670910872, 'eval_precision_weighted': 0.7809703442040479, 'eval_recall_weighted': 0.7904015670910872, 'eval_recall_micro': 0.7904015670910872, 'eval_recall_macro': 0.6894700227013568, 'eval_recall_positive': 0.4939271255060729, 'eval_recall_negative': 0.8850129198966409, 'eval_f1_weighted': 0.7845458744686319, 'eval_f1_micro': 0.7904015670910872, 'eval_f1_macro': 0.6988250408010233, 'eval_f1_positive': 0.5327510917030568, 'eval_f1_negative': 0.8648989898989898, 'eval_runtime': 1.6264, 'eval_samples_per_second': 627.754, 'eval_steps_per_second': 39.35, 'epoch': 7.0}


# **Hing-MBERT Test results**

In [33]:
test_results_mbert = trainer.predict(mbert_dataset['test'])

In [34]:
test_results_mbert

PredictionOutput(predictions=array([[ 1.3126131 , -1.4797031 ],
       [ 0.7133658 , -0.9631423 ],
       [ 0.97357076, -1.2186877 ],
       ...,
       [-1.0652889 ,  1.1249665 ],
       [-0.9574023 ,  1.2268711 ],
       [ 0.6241159 , -0.8929947 ]], dtype=float32), label_ids=array([0, 0, 0, ..., 1, 0, 0]), metrics={'test_loss': 0.6339163780212402, 'test_accuracy': 0.7698334965719883, 'test_precision_weighted': 0.7634071049944814, 'test_recall_weighted': 0.7698334965719883, 'test_recall_micro': 0.7698334965719883, 'test_recall_macro': 0.6717692412306855, 'test_recall_positive': 0.4817813765182186, 'test_recall_negative': 0.8617571059431525, 'test_f1_weighted': 0.766264403370262, 'test_f1_micro': 0.7698334965719884, 'test_f1_macro': 0.6766971596888446, 'test_f1_positive': 0.5031712473572939, 'test_f1_negative': 0.8502230720203953, 'test_runtime': 1.6803, 'test_samples_per_second': 607.617, 'test_steps_per_second': 38.088})

In [35]:
test_results_mbert.predictions.argmax(axis=1)

array([0, 0, 0, ..., 1, 1, 0])

In [36]:
test_df['skip_eng_hingMbert'] = test_results_mbert.predictions.argmax(axis=1)

In [37]:
print(test_results_mbert.metrics)

{'test_loss': 0.6339163780212402, 'test_accuracy': 0.7698334965719883, 'test_precision_weighted': 0.7634071049944814, 'test_recall_weighted': 0.7698334965719883, 'test_recall_micro': 0.7698334965719883, 'test_recall_macro': 0.6717692412306855, 'test_recall_positive': 0.4817813765182186, 'test_recall_negative': 0.8617571059431525, 'test_f1_weighted': 0.766264403370262, 'test_f1_micro': 0.7698334965719884, 'test_f1_macro': 0.6766971596888446, 'test_f1_positive': 0.5031712473572939, 'test_f1_negative': 0.8502230720203953, 'test_runtime': 1.6803, 'test_samples_per_second': 607.617, 'test_steps_per_second': 38.088}


# **Hing BERT Training**

In [38]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [39]:
## dataset pre processing and alignment
# dataset = load_dataset('csv', data_files={'train': "data/final_train.csv",
#                                               'val':"data/final_val.csv",'test':"data/final_test.csv"})

bert_tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/hing-bert")
bert_dataset = dataset.map(
    lambda example: bert_tokenizer(example['transliterated_skip_eng'], max_length=97, padding='max_length', truncation=True),
    batched=True,
    batch_size=64
)


bert_dataset = bert_dataset.remove_columns(["clean_text","language_tags","transliterated_skip_eng"])
bert_dataset.set_format("torch")

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

In [40]:
## model training
bert_model = AutoModelForSequenceClassification.from_pretrained("l3cube-pune/hing-bert")

bert_model.resize_token_embeddings(len(bert_tokenizer))
arguments = TrainingArguments(
    output_dir="sample_Hing_Bert_trainer_5k_skip_eng",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    optim = 'adamw_torch',
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=2e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model='eval_f1_weighted',  # Define the metric for early stopping
    greater_is_better=True,
    save_total_limit=1,
    seed=224,
    report_to="wandb"
)

bert_trainer = WeightedLossTrainer(
    model=bert_model,
    args=arguments,
    train_dataset=bert_dataset['train'],
    eval_dataset=bert_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=bert_tokenizer,
    compute_metrics=compute_metrics
)

bert_trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=0.01))
bert_trainer.add_callback(LoggingCallback("sample_Hing_Bert_trainer_5k_skip_eng/log.jsonl"))

config.json:   0%|          | 0.00/716 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/hing-bert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [41]:
# train the model
wandb.init(project="rh3", name="Hing_Bert_skip_eng2")
bert_trainer.train()

eval/accuracy,▆▃██▅▁▆█
eval/f1_macro,▁▇▇██▇██
eval/f1_micro,▆▃██▅▁▆█
eval/f1_negative,█▃██▅▁▆█
eval/f1_positive,▁█▇█████
eval/f1_weighted,▁▅▇█▇▄▇█
eval/loss,▄▂▁▂▃▄█▂
eval/precision_weighted,▁▇▇███▇█
eval/recall_macro,▁▇▆▇███▇
eval/recall_micro,▆▃██▅▁▆█
eval/recall_negative,█▂▆▆▄▁▄▆


Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,Recall Micro,Recall Macro,Recall Positive,Recall Negative,F1 Weighted,F1 Micro,F1 Macro,F1 Positive,F1 Negative
1,No log,0.692237,0.704212,0.650234,0.704212,0.704212,0.516846,0.153846,0.879845,0.669136,0.704212,0.509784,0.201058,0.818510
2,No log,0.684732,0.617042,0.718744,0.617042,0.617042,0.621991,0.631579,0.612403,0.644081,0.617042,0.575902,0.443812,0.707991
3,No log,0.645223,0.632713,0.718862,0.632713,0.632713,0.624057,0.607287,0.640827,0.657641,0.632713,0.585061,0.444444,0.725677
4,No log,0.599815,0.694417,0.743549,0.694417,0.694417,0.661998,0.599190,0.724806,0.710919,0.694417,0.634634,0.486842,0.782427
5,No log,0.599252,0.715965,0.738343,0.715965,0.715965,0.651401,0.526316,0.776486,0.725094,0.715965,0.639179,0.472727,0.805630
6,No log,0.603704,0.731636,0.749183,0.731636,0.731636,0.665872,0.538462,0.793282,0.738957,0.731636,0.655085,0.492593,0.817577
7,No log,0.621793,0.745348,0.752501,0.745348,0.745348,0.666646,0.514170,0.819121,0.748635,0.745348,0.662003,0.494163,0.829843
8,No log,0.686466,0.729677,0.750516,0.729677,0.729677,0.668714,0.550607,0.786822,0.738110,0.729677,0.655806,0.496350,0.815261
9,No log,0.793310,0.763957,0.751987,0.763957,0.763957,0.651354,0.433198,0.869509,0.756741,0.763957,0.659235,0.470330,0.848141
10,No log,0.747543,0.658178,0.744941,0.658178,0.658178,0.661527,0.668016,0.655039,0.681547,0.658178,0.614978,0.486009,0.743947


TrainOutput(global_step=480, training_loss=0.5621100743611653, metrics={'train_runtime': 149.1834, 'train_samples_per_second': 205.117, 'train_steps_per_second': 3.218, 'train_loss': 0.5621100743611653, 'epoch': 10.0})

In [42]:
bert_eval_results = bert_trainer.evaluate()
print(bert_eval_results)

{'eval_loss': 0.7933104634284973, 'eval_accuracy': 0.7639569049951028, 'eval_precision_weighted': 0.7519866475599134, 'eval_recall_weighted': 0.7639569049951028, 'eval_recall_micro': 0.7639569049951028, 'eval_recall_macro': 0.651353712247225, 'eval_recall_positive': 0.4331983805668016, 'eval_recall_negative': 0.8695090439276486, 'eval_f1_weighted': 0.7567411128388635, 'eval_f1_micro': 0.7639569049951028, 'eval_f1_macro': 0.6592354085737828, 'eval_f1_positive': 0.47032967032967027, 'eval_f1_negative': 0.8481411468178953, 'eval_runtime': 1.6279, 'eval_samples_per_second': 627.198, 'eval_steps_per_second': 39.315, 'epoch': 10.0}


# **Test results for Hing-BERT**

In [43]:
bert_results = bert_trainer.predict(bert_dataset['test'])
test_df['skip_eng_hingBert'] = bert_results.predictions.argmax(axis=1)
print(bert_results.metrics)

{'test_loss': 0.8183923959732056, 'test_accuracy': 0.7404505386875612, 'test_precision_weighted': 0.7351417241982867, 'test_recall_weighted': 0.7404505386875612, 'test_recall_micro': 0.7404505386875612, 'test_recall_macro': 0.6358498362782328, 'test_recall_positive': 0.4331983805668016, 'test_recall_negative': 0.838501291989664, 'test_f1_weighted': 0.7376320507141493, 'test_f1_micro': 0.7404505386875612, 'test_f1_macro': 0.6386091732482766, 'test_f1_positive': 0.4467640918580375, 'test_f1_negative': 0.8304542546385156, 'test_runtime': 1.6726, 'test_samples_per_second': 610.429, 'test_steps_per_second': 38.264}


# **Muril cased training**

In [44]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [45]:
muril_tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
print(len(muril_tokenizer))

muril_dataset = dataset.map(
    lambda example: muril_tokenizer(example['transliterated_skip_eng'], max_length=97, padding='max_length', truncation=True),
    batched=True,
    batch_size=64
)

muril_dataset = muril_dataset.remove_columns(["clean_text","language_tags","transliterated_skip_eng"])
muril_dataset.set_format("torch")

muril_model = AutoModelForSequenceClassification.from_pretrained("google/muril-base-cased",num_labels=2)


arguments = TrainingArguments(
    output_dir="sample_Hing_muril_trainer_5k_skip_eng",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy= "epoch",
    learning_rate=3e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.001,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model='eval_f1_weighted',  # Define the metric for early stopping
    greater_is_better=True,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224,
    report_to="wandb"
)

muril_trainer = WeightedLossTrainer(
    model=muril_model,
    args=arguments,
    train_dataset=muril_dataset['train'],
    eval_dataset=muril_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=muril_tokenizer,
    compute_metrics=compute_metrics
)

muril_trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=0.01))
muril_trainer.add_callback(LoggingCallback("sample_Hing_muril_trainer_5k_skip_eng/log.jsonl"))

tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

197285


Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/953M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [46]:
wandb.init(project="rh3", name="muril_skip_eng")
muril_trainer.train()

eval/accuracy,▅▁▂▅▆▆▇▆█▃█
eval/f1_macro,▁▄▄▇▇████▆█
eval/f1_micro,▅▁▂▅▆▆▇▆█▃█
eval/f1_negative,▇▁▂▅▆▆▇▆█▃█
eval/f1_positive,▁▇▇█▇███▇█▇
eval/f1_weighted,▃▁▂▅▆▇▇▇█▃█
eval/loss,▄▄▃▁▁▁▂▄█▆█
eval/precision_weighted,▁▆▆▇▇████▇█
eval/recall_macro,▁▆▆█▇███▇█▇
eval/recall_micro,▅▁▂▅▆▆▇▆█▃█
eval/recall_negative,█▁▂▄▅▆▆▆█▂█


Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,Recall Micro,Recall Macro,Recall Positive,Recall Negative,F1 Weighted,F1 Micro,F1 Macro,F1 Positive,F1 Negative
1,No log,0.674304,0.774731,0.775354,0.774731,0.774731,0.694295,0.538462,0.850129,0.775039,0.774731,0.693760,0.536290,0.851229
2,No log,0.609396,0.663075,0.790018,0.663075,0.663075,0.715754,0.817814,0.613695,0.687213,0.663075,0.637132,0.540107,0.734158
3,0.640500,0.559569,0.719882,0.807075,0.719882,0.719882,0.750466,0.809717,0.691214,0.739251,0.719882,0.686088,0.583090,0.789086
4,0.640500,0.671266,0.796278,0.781464,0.796278,0.796278,0.676806,0.445344,0.908269,0.784736,0.796278,0.692573,0.514019,0.871128
5,0.640500,0.686068,0.786484,0.776805,0.786484,0.786484,0.684129,0.485830,0.882429,0.780519,0.786484,0.693196,0.524017,0.862374


TrainOutput(global_step=960, training_loss=0.5420135974884033, metrics={'train_runtime': 124.0919, 'train_samples_per_second': 246.592, 'train_steps_per_second': 15.472, 'train_loss': 0.5420135974884033, 'epoch': 5.0})

In [47]:
muril_eval_results = muril_trainer.evaluate()
print(muril_eval_results)

{'eval_loss': 0.6712656021118164, 'eval_accuracy': 0.7962781586679726, 'eval_precision_weighted': 0.7814640381506902, 'eval_recall_weighted': 0.7962781586679726, 'eval_recall_micro': 0.7962781586679726, 'eval_recall_macro': 0.6768064317023925, 'eval_recall_positive': 0.44534412955465585, 'eval_recall_negative': 0.9082687338501292, 'eval_f1_weighted': 0.784735949977002, 'eval_f1_micro': 0.7962781586679726, 'eval_f1_macro': 0.6925731623991014, 'eval_f1_positive': 0.5140186915887851, 'eval_f1_negative': 0.8711276332094176, 'eval_runtime': 1.6271, 'eval_samples_per_second': 627.483, 'eval_steps_per_second': 39.333, 'epoch': 5.0}


In [48]:
muril_results = muril_trainer.predict(muril_dataset['test'])
test_df['skip_eng_muril'] = muril_results.predictions.argmax(axis=1)
print(muril_results.metrics)

{'test_loss': 0.668336033821106, 'test_accuracy': 0.7923604309500489, 'test_precision_weighted': 0.7773355514732374, 'test_recall_weighted': 0.7923604309500489, 'test_recall_micro': 0.7923604309500489, 'test_recall_macro': 0.6728441557082928, 'test_recall_positive': 0.44129554655870445, 'test_recall_negative': 0.9043927648578811, 'test_f1_weighted': 0.7810300612522226, 'test_f1_micro': 0.7923604309500488, 'test_f1_macro': 0.6877315482716833, 'test_f1_positive': 0.5069767441860465, 'test_f1_negative': 0.8684863523573201, 'test_runtime': 1.6697, 'test_samples_per_second': 611.49, 'test_steps_per_second': 38.33}


## XLMR training

In [49]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [50]:
xlmr_tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")

xlmr_dataset = dataset.map(
    lambda example: xlmr_tokenizer(example['transliterated_skip_eng'], max_length=194, padding='max_length', truncation=True),
    batched=True,
    batch_size=16
)

xlmr_dataset = xlmr_dataset.remove_columns(["clean_text","language_tags","transliterated_skip_eng"])
xlmr_dataset.set_format("torch")

xlmr_model = AutoModelForSequenceClassification.from_pretrained("FacebookAI/xlm-roberta-base",num_labels=2)

arguments = TrainingArguments(
    output_dir="sample_Hing_xlmr_trainer_5k_skip_eng",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy= "epoch",
    learning_rate=2e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.001,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model='eval_f1_weighted',  # Define the metric for early stopping
    greater_is_better=True,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224,
    report_to="wandb"
)

xlmr_trainer = WeightedLossTrainer(
    model=xlmr_model,
    args=arguments,
    train_dataset=xlmr_dataset['train'],
    eval_dataset=xlmr_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=xlmr_tokenizer,
    compute_metrics=compute_metrics
)

xlmr_trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=0.01))
xlmr_trainer.add_callback(LoggingCallback("sample_Hing_xlmr_trainer_5k_skip_eng/log.jsonl"))

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [51]:
import torch
torch.cuda.empty_cache()

In [52]:
wandb.init(project="rh3", name="xlmr_skip_eng")
xlmr_trainer.train()

eval/accuracy,▇▁▄█▇█
eval/f1_macro,█▁▇███
eval/f1_micro,▇▁▄█▇█
eval/f1_negative,▇▁▄███
eval/f1_positive,▃▄█▁▂▁
eval/f1_weighted,▇▁▅███
eval/loss,▇▄▁▇█▇
eval/precision_weighted,▁▄█▂▁▂
eval/recall_macro,▃▅█▁▂▁
eval/recall_micro,▇▁▄█▇█
eval/recall_negative,▇▁▃█▇█


Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,Recall Micro,Recall Macro,Recall Positive,Recall Negative,F1 Weighted,F1 Micro,F1 Macro,F1 Positive,F1 Negative
1,No log,0.689594,0.758080,0.574686,0.758080,0.758080,0.500000,0.000000,1.000000,0.653765,0.758080,0.431198,0.000000,0.862396
2,No log,0.596339,0.758080,0.763876,0.758080,0.758080,0.681935,0.534413,0.829457,0.760761,0.758080,0.677651,0.516634,0.838668
3,0.648000,0.625167,0.756121,0.797662,0.756121,0.756121,0.741288,0.712551,0.770026,0.768777,0.756121,0.706447,0.585691,0.827203
4,0.648000,0.650629,0.759060,0.797282,0.759060,0.759060,0.740470,0.704453,0.776486,0.771021,0.759060,0.707985,0.585859,0.830110
5,0.648000,0.613432,0.731636,0.804870,0.731636,0.731636,0.749948,0.785425,0.714470,0.749353,0.731636,0.693776,0.586103,0.801449
6,0.456800,0.859868,0.787463,0.793919,0.787463,0.787463,0.724746,0.603239,0.846253,0.790335,0.787463,0.718266,0.578641,0.857891
7,0.456800,1.201577,0.790402,0.787585,0.790402,0.790402,0.707388,0.546559,0.868217,0.788909,0.790402,0.710248,0.557851,0.862644
8,0.271800,1.389899,0.794319,0.790501,0.794319,0.794319,0.709972,0.546559,0.873385,0.792242,0.794319,0.714028,0.562500,0.865557
9,0.271800,1.477624,0.789422,0.789133,0.789422,0.789422,0.712255,0.562753,0.861757,0.789277,0.789422,0.712548,0.563895,0.861201
10,0.271800,1.607149,0.785504,0.784043,0.785504,0.785504,0.704158,0.546559,0.861757,0.784753,0.785504,0.705565,0.552147,0.858983


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=1920, training_loss=0.38820021947224936, metrics={'train_runtime': 413.1219, 'train_samples_per_second': 74.07, 'train_steps_per_second': 4.648, 'train_loss': 0.38820021947224936, 'epoch': 10.0})

In [53]:
xlmr_eval_results = xlmr_trainer.evaluate()
print(xlmr_eval_results)

{'eval_loss': 1.389899492263794, 'eval_accuracy': 0.7943192948090108, 'eval_precision_weighted': 0.7905009164530544, 'eval_recall_weighted': 0.7943192948090108, 'eval_recall_micro': 0.7943192948090108, 'eval_recall_macro': 0.709971858686669, 'eval_recall_positive': 0.5465587044534413, 'eval_recall_negative': 0.8733850129198967, 'eval_f1_weighted': 0.7922415290424768, 'eval_f1_micro': 0.7943192948090106, 'eval_f1_macro': 0.7140284891165173, 'eval_f1_positive': 0.5625, 'eval_f1_negative': 0.8655569782330346, 'eval_runtime': 2.9757, 'eval_samples_per_second': 343.11, 'eval_steps_per_second': 21.507, 'epoch': 10.0}


In [54]:
xlmr_results = xlmr_trainer.predict(xlmr_dataset['test'])
test_df['skip_eng_xlmr'] = xlmr_results.predictions.argmax(axis=1)
print(xlmr_results.metrics)

{'test_loss': 1.3355460166931152, 'test_accuracy': 0.801175318315377, 'test_precision_weighted': 0.7982612994967077, 'test_recall_weighted': 0.801175318315377, 'test_recall_micro': 0.801175318315377, 'test_recall_macro': 0.7213853058406302, 'test_recall_positive': 0.5668016194331984, 'test_recall_negative': 0.875968992248062, 'test_f1_weighted': 0.7996127032369509, 'test_f1_micro': 0.801175318315377, 'test_f1_macro': 0.7247492353887199, 'test_f1_positive': 0.5797101449275361, 'test_f1_negative': 0.8697883258499037, 'test_runtime': 3.0034, 'test_samples_per_second': 339.943, 'test_steps_per_second': 21.309}


In [55]:
wandb.finish()
test_df.to_csv('skip_eng_muril_v2.csv',index=False, encoding='utf-8')

eval/accuracy,▄▄▄▄▁▇██▇▇█
eval/f1_macro,▁▇██▇██████
eval/f1_micro,▄▄▄▄▁▇██▇▇█
eval/f1_negative,█▅▄▄▁▇███▇█
eval/f1_positive,▁▇█████████
eval/f1_weighted,▁▆▇▇▆██████
eval/loss,▂▁▁▁▁▃▅▆▇█▆
eval/precision_weighted,▁▇████▇██▇█
eval/recall_macro,▁▆███▇▇▇▇▇▇
eval/recall_micro,▄▄▄▄▁▇██▇▇█
eval/recall_negative,█▄▂▃▁▄▅▅▅▅▅


In [56]:
test_df['skip_eng'] = dataset['test']['transliterated_skip_eng']
test_df.to_csv('skip_eng_v2.csv',index=False, encoding='utf-8')